In [11]:
import pandas as pd


In [10]:
import sys
sys.path


['C:\\Users\\star5\\EE101',
 'C:\\python\\python311.zip',
 'C:\\python\\DLLs',
 'C:\\python\\Lib',
 'C:\\python',
 'C:\\Users\\star5\\AppData\\Roaming\\Python\\Python311\\site-packages',
 'C:\\python\\Lib\\site-packages',
 'C:\\python\\Lib\\site-packages\\vboxapi-1.0-py3.11.egg',
 'C:\\python\\Lib\\site-packages\\win32',
 'C:\\python\\Lib\\site-packages\\win32\\lib',
 'C:\\python\\Lib\\site-packages\\Pythonwin']

In [13]:
kk=pd.read_csv(filepath_or_buffer=r'country-population.CSV')
print(kk.head(5))

                  Country Name Country Code     Indicator Name Indicator Code  \
0                        Aruba          ABW  Population, total    SP.POP.TOTL   
1  Africa Eastern and Southern          AFE  Population, total    SP.POP.TOTL   
2                  Afghanistan          AFG  Population, total    SP.POP.TOTL   
3   Africa Western and Central          AFW  Population, total    SP.POP.TOTL   
4                       Angola          AGO  Population, total    SP.POP.TOTL   

          1960         1961         1962         1963         1964  \
0      54608.0      55811.0      56682.0      57475.0      58178.0   
1  130692579.0  134169237.0  137835590.0  141630546.0  145605995.0   
2    8622466.0    8790140.0    8969047.0    9157465.0    9355514.0   
3   97256290.0   99314028.0  101445032.0  103667517.0  105959979.0   
4    5357195.0    5441333.0    5521400.0    5599827.0    5673199.0   

          1965  ...         2014         2015         2016         2017  \
0      58782.0  .